In [1]:
import sys
sys.path.insert(0,"/home/jeandet/Documents/prog/CDFpp/build")

import numpy as np
import pycdfpp
pycdfpp.__file__

'/home/jeandet/Documents/prog/CDFpp/build/pycdfpp/__init__.py'

In [2]:
pycdfpp.CompressionType.delta_plus_zstd_compression

<CompressionType.delta_plus_zstd_compression: 48>

In [3]:
f=pycdfpp.load("/home/jeandet/Downloads/mms1_scm_srvy_l2_scsrvy_20220811_v2.2.0.cdf")

In [4]:
for name,v in f.items():
    if v.type in ( pycdfpp.DataType.CDF_TIME_TT2000, pycdfpp.DataType.CDF_INT1, pycdfpp.DataType.CDF_INT2, pycdfpp.DataType.CDF_INT4, pycdfpp.DataType.CDF_INT8):
        v.compression = pycdfpp.CompressionType.delta_plus_zstd_compression
    if v.type in ( pycdfpp.DataType.CDF_FLOAT, pycdfpp.DataType.CDF_REAL4, pycdfpp.DataType.CDF_DOUBLE, pycdfpp.DataType.CDF_REAL8):
        if len(v)>1:
            v.compression = pycdfpp.CompressionType.float_zstd_compression

pycdfpp.save(f, "/tmp/test.cdf")

True

In [7]:
for input_file in ('mms1_fpi_brst_l2_des-dist_20210701000343_v3.4.0.cdf', 
                   'mms1_fpi_brst_l2_des-dist_20230602042743_v3.4.0.cdf',
                   'solo_l2_rpw-lfr-surv-swf-b_20220221_v02.cdf',
                   'tha_l2_scm_20161228_v01.cdf'
                  ):
    f=pycdfpp.load(f"/home/jeandet/Downloads/{input_file}")
    for name,v in f.items():
        if v.type in ( pycdfpp.DataType.CDF_TIME_TT2000, ):
            v.compression = pycdfpp.CompressionType.delta_plus_zstd_compression
        if v.type in (pycdfpp.DataType.CDF_INT1, pycdfpp.DataType.CDF_INT2, pycdfpp.DataType.CDF_INT4, pycdfpp.DataType.CDF_INT8):
            v.compression = pycdfpp.CompressionType.gzip_compression
        if v.type in ( pycdfpp.DataType.CDF_FLOAT, pycdfpp.DataType.CDF_REAL4, pycdfpp.DataType.CDF_DOUBLE, pycdfpp.DataType.CDF_REAL8):
            if len(v)>10:
                v.compression = pycdfpp.CompressionType.gzip_compression

    pycdfpp.save(f, f"/tmp/{input_file}")

In [4]:
%timeit pycdfpp.save(f, "/tmp/test.cdf")

1.43 s ± 738 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit pycdfpp.load("/tmp/test.cdf", lazy_load=False)

477 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
pycdfpp.load("/home/jeandet/Downloads/solo_l2_rpw-lfr-surv-swf-b_20220221_v02.cdf")

CDF:
  version: 3.9.0
  majority: column
  compression: None

Attributes:
  Logical_source_description: "Solar Orbiter Radio/Plasma Wave, LFR L2 magnetic parameters"
  CALIBRATION_VERSION: [ [ "20190123171020", "V20200428000000" ] ]
  Free_field: " "
  Rules_of_use: "Data provided are publicly available. If used in presentations or publications, please acknowledge Solar Orbiter/RPW Investigation (M. Maksimovic, PIs). Please read carefully the caveats attached to the data. For reporting purposes, we request bibliography information for any publication, etc., using these data. Please send information on the use of this data to: M. Maksimovic (PI)."
  Parents: [ [ "CDF>solo_L1R_rpw-lfr-surv-swf-b-cdag_20220221_V02", "CDF>solo_HK_rpw-lfr_20220221_V03", "CDF>solo_HK_rpw-pdu_20220221_V03" ] ]
  SOOP_TYPE: " "
  Dataset_ID: "SOLO_L2_RPW-LFR-SURV-SWF-B"
  Data_type: "H0>High Resolution data"
  ACCESS_URL: "https://rpw.lesia.obspm.fr/roc/data/pub/solo/rpw/data/L2/lfr_wf_b/2022/02/solo_L2_rpw-lf

In [5]:
for name,v in f.items():
    if np.prod(v.shape) > 100:
        v.compression = pycdfpp.CompressionType.zstd_compression
pycdfpp.save(f, "/tmp/test_full_zstd.cdf")

True

In [11]:
np.prod((1,2,3))

6

In [18]:
f2=pycdfpp.load("/tmp/test.cdf")
for name,v in f.items():
    if not np.all(v.values == f2[name].values):
        if not np.array_equal(v.values, f2[name].values, equal_nan=True):
            print(name)


In [12]:
f2["mms1_scm_acb_gse_scsrvy_srvy_l2"].values

array([[ 0.06627767,  0.10891791,  0.0312961 ],
       [ 0.02560943,  0.06024531,  0.0083373 ],
       [ 0.02447548, -0.0096865 ,  0.01616823],
       ...,
       [-0.24406226,  0.07305223,  0.05498237],
       [-0.22405699,  0.06590237,  0.06501136],
       [-0.20598231,  0.0552755 ,  0.07047489]], dtype=float32)

In [13]:
f["mms1_scm_acb_gse_scsrvy_srvy_l2"].values

array([[ 0.06627767,  0.10891791,  0.0312961 ],
       [ 0.02560943,  0.06024531,  0.0083373 ],
       [ 0.02447548, -0.0096865 ,  0.01616823],
       ...,
       [-0.24406226,  0.07305223,  0.05498237],
       [-0.22405699,  0.06590237,  0.06501136],
       [-0.20598231,  0.0552755 ,  0.07047489]], dtype=float32)

In [17]:
np.array_equal(f["mms1_scm_acb_gse_scsrvy_srvy_l2"].values, f2["mms1_scm_acb_gse_scsrvy_srvy_l2"].values, equal_nan=True)

True

In [8]:
f["tha_scf_gse"].len()

AttributeError: 'pycdfpp._pycdfpp.Variable' object has no attribute 'len'

In [2]:
pycdfpp.load("../tests/resources/a_cdf.cdf")

CDF:
  version: 3.9.0
  majority: row
  compression: None

Attributes:
  attr: "a cdf text attribute"
  attr_float: [ [ [ 1, 2, 3 ], [ 4, 5, 6 ] ] ]
  attr_int: [ [ [ 1, 2, 3 ] ] ]
  attr_multi: [ [ [ 1, 2 ], [ 2, 3 ], "hello" ] ]
  empty: [ [  ] ]
  epoch: [ [ [ 1970-01-01T00:00:00.000000000, 1970-06-30T00:00:00.000000000, 1970-12-27T00:00:00.000000000, 1971-06-25T00:00:00.000000000, 1971-12-22T00:00:00.000000000, 1972-06-19T00:00:00.000000000, 1972-12-16T00:00:00.000000000, 1973-06-14T00:00:00.000000000, 1973-12-11T00:00:00.000000000, 1974-06-09T00:00:00.000000000, 1974-12-06T00:00:00.000000000 ] ] ]
  epoch16: [ [ [ 1970-01-01T00:00:00.000000000, 1970-06-30T00:00:00.000000000, 1970-12-27T00:00:00.000000000, 1971-06-25T00:00:00.000000000, 1971-12-22T00:00:00.000000000, 1972-06-19T00:00:00.000000000, 1972-12-16T00:00:00.000000000, 1973-06-14T00:00:00.000000000, 1973-12-11T00:00:00.000000000, 1974-06-09T00:00:00.000000000, 1974-12-06T00:00:00.000000000 ] ] ]
  tt2000: [ [ [ 1970-01-01T